In [1]:
import sys
sys.path.append("../")
import numpy as np
import matplotlib.pyplot as plt
import scripts.theodolite_utils as theodo_u
import scripts.theodolite_plot_function as theodo_p
import scripts.groundtruth_utils as theodo_g
from scipy.spatial.transform import Rotation as R
from numpy import linalg
import importlib
theodo_u = importlib.reload(theodo_u)
theodo_p = importlib.reload(theodo_p)

In [ ]:
theodo_u = importlib.reload(theodo_u)
np.random.seed(10)

num_samples_MC_sensor = 10   ## Number sample for uncertainty propagation to the sensor

In [ ]:
theodo_g = importlib.reload(theodo_g)
theodo_u = importlib.reload(theodo_u)

# Compute covariance of all measurements for the dataset selected and model chosen
Data_uncertainty_total = []

P1 = np.array([3.28, -0.41, -0.03, 1.0])
C1 = np.array([[1.8098167474423251e-16, 2.045312300121645e-15, 3.063801387204384e-12],
               [2.045312300121645e-15, 1.824803527827509e-12, 2.724552781709242e-09],
               [3.063801387204384e-12, 2.724552781709242e-09, 4.09486576064592e-06]])
P2 = np.array([4.01, -3.88e-05, -0.33, 1])
C2 = np.array([[6.8626358612273644e-16, -9.859519265046416e-16, -1.4257721642435706e-12],
               [-9.859519265046416e-16, 2.017399767238251e-12, 2.8049731567599196e-09],
               [-1.4257721642435706e-12, 2.8049731567599196e-09, 3.919312566087246e-06]])
P3 = np.array([3.01, 0.29, -0.37, 1.0])
C3 = np.array([[1.6665677076045173e-16, 6.924811486955276e-18, -9.056358581389611e-13],
               [6.924811486955276e-18, 3.9603820338900725e-14, 3.3368162754862885e-10],
               [-9.056358581389611e-13, 3.3368162754862885e-10, 3.807330902205233e-06]])

T_used = np.array([[-0.0000000,  0.0000000, -1.0000000, 5],
                   [0.7071068, -0.7071068, -0.0000000,-2],
                   [-0.7071068, -0.7071068, -0.0000000,1],
                   [0,0,0,1]])

P1_init = T_used@P1
P2_init = T_used@P2
P3_init = T_used@P3

value_cov = 0.02
p1_cov_x = value_cov
p1_cov_y = value_cov
p1_cov_z = value_cov
p2_cov_x = value_cov
p2_cov_y = value_cov
p2_cov_z = value_cov
p3_cov_x = value_cov
p3_cov_y = value_cov
p3_cov_z = value_cov

C1_init = np.linalg.diag([np.random.normal(0, p1_cov_x, 1), np.random.normal(0, p1_cov_y, 1), np.random.normal(0, p1_cov_z, 1)])
C2_init = np.linalg.diag([np.random.normal(0, p2_cov_x, 1), np.random.normal(0, p2_cov_y, 1), np.random.normal(0, p2_cov_z, 1)])
C3_init = np.linalg.diag([np.random.normal(0, p3_cov_x, 1), np.random.normal(0, p3_cov_y, 1), np.random.normal(0, p3_cov_z, 1)])

q1_corrected = theodo_g.return_point_from_covariance(C1, P1, num_samples_MC_sensor)
q2_corrected = theodo_g.return_point_from_covariance(C2, P2, num_samples_MC_sensor)
q3_corrected = theodo_g.return_point_from_covariance(C3, P3, num_samples_MC_sensor)

p1_corrected = theodo_g.return_point_from_covariance(C1_init, P1_init, num_samples_MC_sensor)
p2_corrected = theodo_g.return_point_from_covariance(C2_init, P2_init, num_samples_MC_sensor)
p3_corrected = theodo_g.return_point_from_covariance(C3_init, P3_init, num_samples_MC_sensor)


P_sensor = theodo_g.chose_sensor_before_ptp_uncertainty(file_path, Sensor, Gps_reference_chosen, num_samples_MC_sensor)
Pose_sensor_MC = []
Dist_prism_MC = []
for i_mc,j_mc,k_mc in zip(MC_1_sorted,MC_2_sorted,MC_3_sorted):
    Pose_sensor = []
    Dist_prism_MC.append([i_mc[0],np.linalg.norm(i_mc[1]-j_mc[1]),np.linalg.norm(i_mc[1]-k_mc[1]),np.linalg.norm(k_mc[1]-j_mc[1])])
    p1_corrected = theodo_g.return_point_from_covariance(i_mc[2],i_mc[1], num_samples_MC_sensor)
    p2_corrected = theodo_g.return_point_from_covariance(j_mc[2],j_mc[1], num_samples_MC_sensor)
    p3_corrected = theodo_g.return_point_from_covariance(k_mc[2],k_mc[1], num_samples_MC_sensor)
    try:
        for i_estimated,j_estimated,k_estimated,i_truth,j_truth,k_truth in zip(p1_corrected,p2_corrected,p3_corrected, P_sensor[0], P_sensor[1],P_sensor[2]):
            Q = np.array([i_estimated, j_estimated, k_estimated]).T
            Q = np.concatenate((Q, np.array([[1,1,1]])), axis=0)
            P = np.array([i_truth, j_truth, k_truth]).T
            P = np.concatenate((P, np.array([[1,1,1]])), axis=0)
            T = theodo_u.point_to_point_minimization(P, Q)
            Pose_sensor.append(T)
        mu_T, cov_matrix_T, mu_A, cov_matrix_A = theodo_g.find_noise_list_tf(Pose_sensor)
        Pose_sensor_MC.append([i_estimated[0], mu_T, cov_matrix_T, mu_A, cov_matrix_A])
    except:
        print("Exception extraction !")

# if save_MC_sensor:
#     if Sensor=="GNSS":
#         # theodo_u.save_MC_pose_sorted(Pose_sensor_MC, file_path + "uncertainty/sensor/"+Sensor+"_"+str(Gps_reference_chosen)+"_"
#         #                                      +str(model_chosen[0])+"_"+str(model_chosen[2])+"_"+str(model_chosen[4])
#         #                                  +"_"+str(model_chosen[6])+"_"+str(model_chosen[8])+".csv")
#         #theodo_u.save_MC_dist_sorted(Dist_prism_MC, file_path + "uncertainty/sensor/"+Sensor+"_DIST_"+str(Gps_reference_chosen)+"_"
#          #                                    +str(model_chosen[0])+"_"+str(model_chosen[2])+"_"+str(model_chosen[4])
#          #                                +"_"+str(model_chosen[6])+"_"+str(model_chosen[8])+".csv")
#     else:
#         # theodo_u.save_MC_pose_sorted(Pose_sensor_MC, file_path + "uncertainty/sensor/"+Sensor+"_"
#         #                                      +str(model_chosen[0])+"_"+str(model_chosen[2])+"_"+str(model_chosen[4])
#         #                                  +"_"+str(model_chosen[6])+"_"+str(model_chosen[8])+".csv")
#         #theodo_u.save_MC_dist_sorted(Dist_prism_MC, file_path + "uncertainty/sensor/"+Sensor+"_DIST_"
#          #                                    +str(model_chosen[0])+"_"+str(model_chosen[2])+"_"+str(model_chosen[4])
#          #                                +"_"+str(model_chosen[6])+"_"+str(model_chosen[8])+".csv")

# print("Interpolation finished !")
